## Setup

In [ ]:
from src.data.setup import download_and_extract

download_and_extract(
    url="https://drive.google.com/uc?id=1VAc_MFUAlrtqZanxwYRRtGYVLFunKABf",
    zip_path="data.zip",
    extract_dir="data",
    verbose=True,
    remove_zip=True
)

## Train DT

In [ ]:
from src.models.decision_tree.train import main as train_decision_tree

train_decision_tree()

## Launch game

In [ ]:
from src.main import main

mode = "dt" # choices: "human", "pc", "dt", "ht", "ct"
main(mode)

Loaded Decision Tree (acc: 0.893)


: 